In [8]:
import tensorflow as tf
import keras
import tensorflow_datasets as tfds

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [9]:
imdb, info=tfds.load('imdb_reviews', with_info=True, as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.38WA2E_1.0.0/imdb_reviews-train.tfrecor…

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.38WA2E_1.0.0/imdb_reviews-test.tfrecord…

Generating unsupervised examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.38WA2E_1.0.0/imdb_reviews-unsupervised.…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [10]:
info

tfds.core.DatasetInfo(
    name='imdb_reviews',
    full_name='imdb_reviews/plain_text/1.0.0',
    description="""
    Large Movie Review Dataset. This is a dataset for binary sentiment
    classification containing substantially more data than previous benchmark
    datasets. We provide a set of 25,000 highly polar movie reviews for training,
    and 25,000 for testing. There is additional unlabeled data for use as well.
    """,
    config_description="""
    Plain text
    """,
    homepage='http://ai.stanford.edu/~amaas/data/sentiment/',
    data_dir='/root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0',
    file_format=tfrecord,
    download_size=80.23 MiB,
    dataset_size=129.83 MiB,
    features=FeaturesDict({
        'label': ClassLabel(shape=(), dtype=int64, num_classes=2),
        'text': Text(shape=(), dtype=string),
    }),
    supervised_keys=('text', 'label'),
    disable_shuffling=False,
    nondeterministic_order=False,
    splits={
        'test': <SplitInfo num_e

In [11]:
imdb

{Split('train'): <_PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>,
 Split('test'): <_PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>,
 Split('unsupervised'): <_PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>}

In [12]:
for i in imdb['train'].take(4):
  print(i)

(<tf.Tensor: shape=(), dtype=string, numpy=b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.">, <tf.Tensor: shape=(), dtype=int64, numpy=0>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on

In [13]:
train_data, test_data= imdb['train'], imdb['test']

In [14]:
train_data

<_PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [15]:
vocab_size=10000
embedding_dim=16
max_len=120
padding_type='pre'
trunc_type='post'

In [16]:
train_sentences=[]
train_labels=[]
test_sentences=[]
test_labels=[]

In [18]:
for s,l in train_data:
  train_sentences.append(s.numpy().decode('utf8'))
  train_labels.append(l.numpy())
for s,l in test_data:
  test_sentences.append(s.numpy().decode('utf8'))
  test_labels.append(l.numpy())

In [19]:
train_sentences[0]

"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."

In [20]:
train_labels[0]

np.int64(0)

In [26]:
train_labels = np.array(train_labels).astype(int)
test_labels = np.array(test_labels).astype(int)

In [28]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [29]:
tokenizer=Tokenizer(num_words=vocab_size, oov_token='<OOV>')
tokenizer.fit_on_texts(train_sentences)

In [30]:
word_index=tokenizer.word_index

In [33]:
sequences=tokenizer.texts_to_sequences(train_sentences)
padded_sequences=pad_sequences(sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

In [34]:
test_sequences=tokenizer.texts_to_sequences(test_sentences)
test_padded_sequences=pad_sequences(test_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

In [35]:
sequences[0]

[12,
 14,
 33,
 425,
 389,
 18,
 90,
 27,
 1,
 9,
 32,
 1365,
 3784,
 40,
 489,
 1,
 199,
 24,
 85,
 154,
 19,
 12,
 215,
 328,
 27,
 66,
 246,
 216,
 9,
 475,
 56,
 66,
 85,
 114,
 98,
 22,
 5710,
 12,
 1334,
 646,
 757,
 12,
 18,
 7,
 33,
 406,
 8294,
 176,
 2472,
 417,
 2,
 89,
 1220,
 137,
 69,
 146,
 52,
 2,
 1,
 7463,
 69,
 228,
 66,
 2948,
 16,
 1,
 2885,
 1,
 1,
 1480,
 4936,
 3,
 39,
 3879,
 115,
 1604,
 17,
 3784,
 14,
 162,
 19,
 4,
 1220,
 899,
 7954,
 9,
 4,
 18,
 13,
 14,
 4126,
 5,
 100,
 145,
 1206,
 11,
 241,
 676,
 13,
 47,
 24,
 99,
 38,
 12,
 7264,
 5469,
 38,
 1365,
 1,
 50,
 400,
 11,
 98,
 1191,
 867,
 141,
 10]

In [36]:
padded_sequences[0]

array([   0,    0,    0,   12,   14,   33,  425,  389,   18,   90,   27,
          1,    9,   32, 1365, 3784,   40,  489,    1,  199,   24,   85,
        154,   19,   12,  215,  328,   27,   66,  246,  216,    9,  475,
         56,   66,   85,  114,   98,   22, 5710,   12, 1334,  646,  757,
         12,   18,    7,   33,  406, 8294,  176, 2472,  417,    2,   89,
       1220,  137,   69,  146,   52,    2,    1, 7463,   69,  228,   66,
       2948,   16,    1, 2885,    1,    1, 1480, 4936,    3,   39, 3879,
        115, 1604,   17, 3784,   14,  162,   19,    4, 1220,  899, 7954,
          9,    4,   18,   13,   14, 4126,    5,  100,  145, 1206,   11,
        241,  676,   13,   47,   24,   99,   38,   12, 7264, 5469,   38,
       1365,    1,   50,  400,   11,   98, 1191,  867,  141,   10],
      dtype=int32)

In [37]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(padded_sequences, train_labels, test_size=0.2, random_state=42)

In [46]:
model=keras.Sequential([
    keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_len),
    keras.layers.Flatten(),
    keras.layers.Dense(6, activation='relu'),
    keras.layers.Dense(2, activation='sigmoid')
])

In [47]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [48]:
history=model.fit(x_train,y_train,epochs=10, validation_data=(x_test,y_test))

Epoch 1/10
1164/1164 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - accuracy: 0.6847 - loss: 0.5478 - val_accuracy: 0.9170 - val_loss: 0.2271
Epoch 2/10
1164/1164 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step - accuracy: 0.9685 - loss: 0.1060 - val_accuracy: 0.9479 - val_loss: 0.1454
Epoch 3/10
1164/1164 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.9973 - loss: 0.0157 - val_accuracy: 0.9535 - val_loss: 0.1561
Epoch 4/10
1164/1164 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - accuracy: 0.9997 - loss: 0.0029 - val_accuracy: 0.9506 - val_loss: 0.1779
Epoch 5/10
1164/1164 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step - accuracy: 1.0000 - loss: 6.9619e-04 - val_accuracy: 0.9535 - val_loss: 0.1903
Epoch 6/10
1164/1164 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 1.0000 - loss: 2.0974e-04 - val_accuracy: 0.9527 - val_loss: 0.2007
Epoch 7/10
1164/1164 ━━━━━━━━━━━━━━━━━━━━ 12s 6ms/step - accuracy: 1.0000 - loss: 1.0184e-04 - val_accuracy: 0.9525 - val_loss: 0.2117
Epoch 8/10
1164/1164 ━━━━━━━━━━━━━━━━━━━━ 11s 7ms/step - accuracy: 1.0000 - 

In [49]:
preds=model.predict(test_padded_sequences)

782/782 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step


In [50]:
preds[:10]

array([[9.2541116e-01, 5.6736910e-01],
       [2.6092235e-02, 9.9903494e-01],
       [9.9935907e-01, 6.3001066e-03],
       [9.9982369e-01, 1.7657934e-03],
       [2.0796850e-03, 9.9991930e-01],
       [3.9768440e-01, 9.4795662e-01],
       [1.6939923e-03, 9.9995160e-01],
       [3.5923297e-04, 9.9999511e-01],
       [2.3531330e-01, 9.8376715e-01],
       [9.1376477e-01, 5.2491653e-01]], dtype=float32)

In [51]:
test_labels[:10]

array([1, 1, 0, 0, 1, 1, 1, 1, 0, 1])

In [52]:
preds=np.argmax(preds, axis=1)

In [53]:
preds[:10]

array([0, 1, 0, 0, 1, 1, 1, 1, 1, 0])

In [54]:
from sklearn.metrics import accuracy_score

In [55]:
accuracy_score(test_labels, preds)

0.80032